# Lecture 03 - Student Notebook

We recommend using Noto for this lecture tutorial, where we've already installed the dependencies of the pymer4 package and statsmodels.

We extended the data with extra features. The feature description is found [here](https://docs.google.com/spreadsheets/d/15UvkrJgTapWispb6tSjMTZh0yJooOsxQ3sWIhKjYM7I/edit?usp=sharing). 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Import the linear regression model class
from pymer4.models import Lm

# Import the lmm model class
from pymer4.models import Lmer

# Data directory
DATA_DIR = "./../../data"

In [2]:
# Parse the aggregated and time series data
df = pd.read_csv('{}/aggregated_extended_fc.csv'.format(DATA_DIR))
df = df.fillna('NaN')
list(df.columns)
display(df)

df_byweek = pd.read_csv('{}/fc_long_extended.csv'.format(DATA_DIR))
display(df_byweek)

,user,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,ch_time_sessions_std,bo_delay_lecture,...,la_weekly_prop_watched_mean,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play,grade,gender,category,year
0,0,1.9,2334.4,2951.8,0.850000,16.8,38.1,1392.858333,790.762032,55068.387500,...,0.245714,0.024286,0.0,0.010000,0.0,0.179203,4.50,NaN,NaN,Y2-2018-19
1,1,3.4,1698.4,9227.8,0.567500,4.0,179.4,3068.720238,1257.504407,-2883.367738,...,0.748868,0.074683,0.0,0.066456,0.0,0.332424,4.50,M,Suisse.Autres,Y2-2018-19
2,2,5.3,2340.6,10801.3,26.562274,94.6,129.2,1750.289268,1024.134043,10027.216667,...,0.354487,0.026667,0.0,0.059915,0.0,0.284407,5.25,M,Suisse.PAM,Y2-2018-19
3,3,2.8,2737.1,8185.5,3.691250,13.5,46.4,20203.590260,656.052901,27596.864484,...,0.370000,0.014286,0.0,0.020000,0.0,0.108774,4.50,F,Suisse.Autres,Y2-2018-19
4,4,2.5,3787.3,7040.0,1.543889,58.4,64.9,3373.908333,1363.320365,-914.633333,...,0.030000,0.000000,0.0,0.020000,0.0,0.199775,4.75,F,France,Y2-2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,293,3.5,8127.5,113.4,0.632304,28.9,20.6,7963.627500,1001.514794,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.034080,5.25,M,France,Y3-2019-20
284,294,2.2,2452.4,4623.1,18.147762,36.4,71.3,3614.055952,853.195566,16834.900000,...,0.140530,0.011111,0.0,0.000000,0.0,0.186649,5.25,F,France,Y3-2019-20
285,296,0.9,1643.2,1932.4,0.000000,0.4,31.2,926.916667,616.918475,-12860.522222,...,0.069231,0.023077,0.0,0.000000,0.0,0.028596,6.00,F,France,Y3-2019-20
286,297,1.4,2718.6,360.3,0.180000,2.0,15.3,346.437500,122.017326,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.032353,5.00,M,Suisse.PAM,Y3-2019-20


,Unnamed: 0,week,user,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,...,la_seek_len_std,la_pause_dur_std,la_time_speeding_up_mean,la_time_speeding_up_std,la_weekly_prop_watched_mean,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play
0,10,0,1,7.0,326.0,15525.0,5.675000,40.0,227.0,1931.285714,...,146.564097,188.175709,65.173554,150.807752,0.600000,0.100000,0.0,0.000000,0.0,0.400749
1,11,1,1,4.0,350.0,8411.0,0.000000,0.0,207.0,2190.250000,...,8.486253,78.639644,47.872928,67.365584,0.800000,0.000000,0.0,0.100000,0.0,0.391304
2,12,2,1,5.0,4577.0,8691.0,0.000000,0.0,167.0,2106.200000,...,63.484419,105.108022,64.533835,81.772612,1.000000,0.000000,0.0,0.000000,0.0,0.359281
3,13,3,1,4.0,259.0,12055.0,0.000000,0.0,239.0,3078.500000,...,31.535282,75.997314,58.085308,86.465139,0.769231,0.000000,0.0,0.153846,0.0,0.359833
4,14,4,1,3.0,480.0,13235.0,0.000000,0.0,197.0,4116.666667,...,10.594150,202.504038,78.057143,140.802708,1.000000,0.285714,0.0,0.285714,0.0,0.390863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2835,5,293,2.0,9315.0,0.0,0.513514,37.0,19.0,4657.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2336,2836,6,293,3.0,86.0,549.0,4.333333,3.0,13.0,211.666667,...,0.000000,116.639044,13.000000,9.000000,0.000000,0.000000,0.0,0.000000,0.0,0.312500
2337,2837,7,293,3.0,3675.0,0.0,0.000000,0.0,41.0,1225.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2338,2838,8,293,5.0,10956.0,0.0,0.000000,0.0,53.0,601.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


### Predicting student performance early on

In this task, we are interested in predicting course grade early on during the semester. This type of information can be useful for an instructor in order to be able to provide intervention to struggling students. We will use again the category as a random effect.
We will need to train a separate model for each week (i.e. predicting after 1 week of the course, after 2 weeks of the course, after 3 weeks, etc.). However, we will use the same equation for all models.

First, we create a dataframe containing information about the user.

In [3]:
# parse the necessary data frames
df_ui = (df.loc[:,['user','grade','gender','category','year']]).copy()

# compute pass/fail label
df_ui['passed'] = df_ui.loc[:,'grade'] >= 4
df_ui.loc[:,'passed'] = df_ui.loc[:,'passed'].replace(True,1)
df_ui.loc[:,'passed'] = df_ui.loc[:,'passed'].replace(False,0)
display(df_ui)

,user,grade,gender,category,year,passed
0,0,4.50,NaN,NaN,Y2-2018-19,1
1,1,4.50,M,Suisse.Autres,Y2-2018-19,1
2,2,5.25,M,Suisse.PAM,Y2-2018-19,1
3,3,4.50,F,Suisse.Autres,Y2-2018-19,1
4,4,4.75,F,France,Y2-2018-19,1
...,...,...,...,...,...,...
283,293,5.25,M,France,Y3-2019-20,1
284,294,5.25,F,France,Y3-2019-20,1
285,296,6.00,F,France,Y3-2019-20,1
286,297,5.00,M,Suisse.PAM,Y3-2019-20,1


Next, we reformat the data frame to contain values by week and user.

In [4]:
df_byuser = df_byweek.sort_values(by=['user', 'week']).reset_index(drop=True)
display(df_byuser)

,Unnamed: 0,week,user,ch_num_sessions,ch_time_in_prob_sum,ch_time_in_video_sum,ch_ratio_clicks_weekend_day,ch_total_clicks_weekend,ch_total_clicks_weekday,ch_time_sessions_mean,...,la_seek_len_std,la_pause_dur_std,la_time_speeding_up_mean,la_time_speeding_up_std,la_weekly_prop_watched_mean,la_weekly_prop_interrupted_mean,la_weekly_prop_interrupted_std,la_weekly_prop_replayed_mean,la_weekly_prop_replayed_std,la_frequency_action_video_play
0,10,0,1,7.0,326.0,15525.0,5.675000,40.0,227.0,1931.285714,...,146.564097,188.175709,65.173554,150.807752,0.600000,0.100000,0.0,0.000000,0.0,0.400749
1,11,1,1,4.0,350.0,8411.0,0.000000,0.0,207.0,2190.250000,...,8.486253,78.639644,47.872928,67.365584,0.800000,0.000000,0.0,0.100000,0.0,0.391304
2,12,2,1,5.0,4577.0,8691.0,0.000000,0.0,167.0,2106.200000,...,63.484419,105.108022,64.533835,81.772612,1.000000,0.000000,0.0,0.000000,0.0,0.359281
3,13,3,1,4.0,259.0,12055.0,0.000000,0.0,239.0,3078.500000,...,31.535282,75.997314,58.085308,86.465139,0.769231,0.000000,0.0,0.153846,0.0,0.359833
4,14,4,1,3.0,480.0,13235.0,0.000000,0.0,197.0,4116.666667,...,10.594150,202.504038,78.057143,140.802708,1.000000,0.285714,0.0,0.285714,0.0,0.390863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2835,5,293,2.0,9315.0,0.0,0.513514,37.0,19.0,4657.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2336,2836,6,293,3.0,86.0,549.0,4.333333,3.0,13.0,211.666667,...,0.000000,116.639044,13.000000,9.000000,0.000000,0.000000,0.0,0.000000,0.0,0.312500
2337,2837,7,293,3.0,3675.0,0.0,0.000000,0.0,41.0,1225.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2338,2838,8,293,5.0,10956.0,0.0,0.000000,0.0,53.0,601.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000


We can now create a model that predicts the exam grade after a specific number of weeks of the course. We will use 5 weeks and 10 weeks.

**Step 1**: We will write a function that aggregates the features for all weeks.

In [5]:
def aggregate_features(df_ui, df_byuser, week_nr):

    df_weeknr = df_byuser[df_byuser['week'] < week_nr]
    df_return = df_weeknr.groupby(['user']).mean()
    df_return['user'] = df_return.index
    
    # Return df with aggregated features
    df_return = df_return.set_index('user').join(df_ui.set_index('user'))
    df_return.reset_index()
    
    return df_return

**Step 2**: We will split the data into a training and test set (20% users in the test set, stratified by pass/fail label). In our case, **data stratification** refers to choosing a sample with the same ratio of pass/fail as the initial dataset, so our training set and our test set are both representative of our original population. If you are interested, you can read more about [stratifying test sets here](https://www.baeldung.com/cs/ml-stratified-sampling).

In [6]:
# perform train/test split
df_week5 = aggregate_features(df_ui, df_byuser, 5)
df_train5, df_test5 = train_test_split(df_week5, test_size=0.2, random_state=0,  stratify=df_week5['passed'])

df_week10 = aggregate_features(df_ui, df_byuser, 10)
df_train10, df_test10 = train_test_split(df_week10, test_size=0.2, random_state=0,  stratify=df_week10['passed'])

**Step 3**: We will now train our model on the training data for 5 and 10 weeks. We will use the following formula:
```grade ~ (1|category) + wa_num_subs_perc_correct```

In [7]:
# Train a multi-regression model for weeks 5 and 10
# Initialize model instance using 1 predictor with random intercepts and slopes
model5 = Lmer("grade ~ (1|category) +  wa_num_subs_perc_correct", data=df_train5, family='gaussian')
model10 = Lmer("grade ~ (1|category) +  wa_num_subs_perc_correct", data=df_train10, family='gaussian')

# Fit the models
print(model5.fit())
print(model10.fit())

Formula: grade~(1|category)+wa_num_subs_perc_correct

Family: gaussian	 Inference: parametric

Number of observations: 187	 Groups: {'category': 5.0}

Log-likelihood: -295.857 	 AIC: 591.714

Random effects:

                 Name    Var    Std
category  (Intercept)  0.072  0.269
Residual               1.349  1.161

No random effect correlations specified

Fixed effects:

                          Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  \
(Intercept)                  3.813   3.404    4.222  0.209   12.154  18.273   
wa_num_subs_perc_correct     0.553  -0.160    1.266  0.364  182.752   1.520   

                          P-val  Sig  
(Intercept)                0.00  ***  
wa_num_subs_perc_correct   0.13       
Formula: grade~(1|category)+wa_num_subs_perc_correct

Family: gaussian	 Inference: parametric

Number of observations: 187	 Groups: {'category': 5.0}

Log-likelihood: -293.932 	 AIC: 587.863

Random effects:

                 Name    Var    Std
category  (Intercept)  0.

**Step 4**: We predict on the test data and check the accuracy.

In [8]:
# predict on the test data for weeks 5, 10
predictions5 = model5.predict(df_test5, verify_predictions=False)
rmse5 = mean_squared_error(df_test5['grade'], predictions5, squared=False)

predictions10 = model10.predict(df_test10, verify_predictions=False)
rmse10 = mean_squared_error(df_test10['grade'], predictions10, squared=False)

print(rmse5)
print(rmse10)

1.1346320524664855
1.1203193591371505


## Your Turn
We are interested in predicting pass/fail (denoted by ```passed``` in the dataframe) instead of the grade.
1. Adjust the equations of ```model5``` and ```model10``` to predict pass/fail instead of the grade.
2. Train the two models, evalute their accuracy on the test data set, and send us the RMSE.

In [9]:
import requests

exec(requests.get("https://courdier.pythonanywhere.com/get-send-code").content)

npt_config = {
    'session_name': 'lecture-03',
    'session_owner': 'mlbd',
    'sender_name': input("Your name: "),
}

Your name:  Vinitra


In [ ]:
# Define the model equations

# YOUR CODE HERE

In [10]:
# Fit the models
print(model5.fit())
print(model10.fit())

# predict on the test data for weeks 5, 10
predictions5 = model5.predict(df_test5, verify_predictions=False)
rmse5 = str(mean_squared_error(df_test5['passed'], predictions5, squared=False))

predictions10 = model10.predict(df_test10, verify_predictions=False)
rmse10 = str(mean_squared_error(df_test10['passed'], predictions10, squared=False))

print(rmse5)
print(rmse10)

# share the RMSEs with us
send(rmse5, 1)
send(rmse10, 2)

Formula: grade~(1|category)+wa_num_subs_perc_correct

Family: gaussian	 Inference: parametric

Number of observations: 187	 Groups: {'category': 5.0}

Log-likelihood: -295.857 	 AIC: 591.714

Random effects:

                 Name    Var    Std
category  (Intercept)  0.072  0.269
Residual               1.349  1.161

No random effect correlations specified

Fixed effects:

                          Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  \
(Intercept)                  3.813   3.404    4.222  0.209   12.154  18.273   
wa_num_subs_perc_correct     0.553  -0.160    1.266  0.364  182.752   1.520   

                          P-val  Sig  
(Intercept)                0.00  ***  
wa_num_subs_perc_correct   0.13       
Formula: grade~(1|category)+wa_num_subs_perc_correct

Family: gaussian	 Inference: parametric

Number of observations: 187	 Groups: {'category': 5.0}

Log-likelihood: -293.932 	 AIC: 587.863

Random effects:

                 Name    Var    Std
category  (Intercept)  0.

<Response [200]>

Extension: if you still have time: can you improve the accuracy of the model by adding more features? Send us an explanation of why you have chosen the specific features along with the RMSE of your model.

In [11]:
# Explain briefly: what features are you adding and why?

exp = """This is an example discussion"""

### Share it with us
send(exp, 3) 

<Response [200]>

In [ ]:
# Define the model equations

# YOUR CODE HERE

In [12]:
# Fit the models
print(model5.fit())
print(model10.fit())

# predict on the test data for weeks 5, 10
predictions5 = model5.predict(df_test5, verify_predictions=False)
rmse5 = str(mean_squared_error(df_test5['passed'], predictions5, squared=False))

predictions10 = model10.predict(df_test10, verify_predictions=False)
rmse10 = str(mean_squared_error(df_test10['passed'], predictions10, squared=False))

print(rmse5)
print(rmse10)

# share the RMSEs with us
send(rmse5, 4)
send(rmse10, 5)

Formula: grade~(1|category)+wa_num_subs_perc_correct

Family: gaussian	 Inference: parametric

Number of observations: 187	 Groups: {'category': 5.0}

Log-likelihood: -295.857 	 AIC: 591.714

Random effects:

                 Name    Var    Std
category  (Intercept)  0.072  0.269
Residual               1.349  1.161

No random effect correlations specified

Fixed effects:

                          Estimate  2.5_ci  97.5_ci     SE       DF  T-stat  \
(Intercept)                  3.813   3.404    4.222  0.209   12.154  18.273   
wa_num_subs_perc_correct     0.553  -0.160    1.266  0.364  182.752   1.520   

                          P-val  Sig  
(Intercept)                0.00  ***  
wa_num_subs_perc_correct   0.13       
Formula: grade~(1|category)+wa_num_subs_perc_correct

Family: gaussian	 Inference: parametric

Number of observations: 187	 Groups: {'category': 5.0}

Log-likelihood: -293.932 	 AIC: 587.863

Random effects:

                 Name    Var    Std
category  (Intercept)  0.

<Response [200]>